In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import regex as re


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/spam.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
df.sample(10)

,Category,Message
5124,ham,He is impossible to argue with and he always t...
5349,ham,I'm home...
1198,ham,He also knows about lunch menu only da. . I know
2829,ham,Wife.how she knew the time of murder exactly
3877,ham,did u get that message
4015,ham,"Sorry, I can't text &amp; drive coherently, se..."
3881,ham,Hi juan. Im coming home on fri hey. Of course ...
3621,ham,I meant as an apology from me for texting you ...
3694,ham,I was about to do it when i texted. I finished...
3126,spam,1st wk FREE! Gr8 tones str8 2 u each wk. Txt N...


In [7]:
df['result'] = df['Category'].apply(lambda x: 1 if x == 'spam' else 0).astype(int)

df['Category'].value_counts() #El dataset no esta balanceado

Category
ham     4825
spam     747
Name: count, dtype: int64

In [18]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z ]',' ',text)
    text = text.strip()
    text = re.sub(r'\s+',' ',text)
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    return text.split()

In [10]:
df['Message'] = df['Message'].apply(preprocess_text)

df

,Category,Message,result
0,ham,[gountiljurongpointcrazyavailableonlyinbugisng...,0
1,ham,[oklarjokingwifuoni],0
2,spam,[freeentryinawklycomptowinfacupfinaltktsstmayt...,1
3,ham,[udunsaysoearlyhorucalreadythensay],0
4,ham,[nahidontthinkhegoestousfhelivesaroundherethough],0
...,...,...,...
5567,spam,[thisisthendtimewehavetriedcontactuuhavewonthe...,1
5568,ham,[willbgoingtoesplanadefrhome],0
5569,ham,[pitywasinmoodforthatsoanyothersuggestions],0
5570,ham,[theguydidsomebitchingbutiactedlikeidbeinteres...,0


In [11]:
df.sample(10)

,Category,Message,result
3473,ham,[ithinkimwaitingforthesamebusinformmewhenyouge...,0
3308,ham,[setaplaceformeinyourheartandnotinyourmindasth...,0
4065,ham,[profyouhavepassedinallthepapersinthissemcongr...,0
1628,spam,[youhavebeenselectedtostayinoftopbritishhotels...,1
3359,ham,[idontrunawayfrmuiwalkslowlyampitkillsmethatud...,0
190,ham,[ijusreachedhomeigobathefirstbutmysisusingnett...,0
956,ham,[sorryinowthencurmsgyarlorsopoorthingbutonlyon...,0
3136,ham,[watmakessomepeopledearerisnotjustdehappinessd...,0
2953,ham,[tellmewhosthispls],0
3560,spam,[urgentwearetryingtocontactutodaysdrawshowstha...,1


In [15]:
download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/arnaldoch/nltk_data...


True

In [14]:
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arnaldoch/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')